# basic-004
https://chatgpt.com/c/6804748b-8800-800a-8422-35b642281b70

In [ ]:
# 📦 1. Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# 📁 2. Load Datasets
full_data = pd.read_csv('full_data.csv', sep=r'\s+')
latest_30 = pd.read_csv('latest_30.csv', sep=r'\s+')

In [ ]:
# 🧹 3. Preprocessing
# Drop unnecessary columns
columns_to_use = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']
full_data = full_data[columns_to_use]
latest_30 = latest_30[columns_to_use]

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
scaled_full = scaler.fit_transform(full_data)
scaled_latest_30 = scaler.transform(latest_30)

In [ ]:
# 🪜 4. Create sequences
def create_sequences(data, input_steps=30, output_steps=100):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+output_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_full, input_steps=30, output_steps=100)

In [ ]:
# 🧪 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 🧠 6. Build the LSTM Model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(30, len(columns_to_use))),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    Dense(len(columns_to_use))
])

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# 🚂 7. Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

In [ ]:
# 🔮 8. Predict Next 100 Candles Using latest_30
latest_30_input = np.expand_dims(scaled_latest_30, axis=0)  # shape: (1, 30, features)
predicted_100 = model.predict(latest_30_input)[0]  # shape: (100, features)

In [ ]:
# 🔄 9. Inverse scale predictions to original values
combined = np.concatenate([scaled_latest_30, predicted_100], axis=0)
combined_inversed = scaler.inverse_transform(combined)
real_30 = combined_inversed[:30]
pred_100 = combined_inversed[30:]

In [ ]:
# 📊 10. Plot 130 candles (30 real + 100 predicted)
plt.figure(figsize=(15, 6))
plt.plot(range(30), real_30[:, 3], label='Real Close', color='blue')      # Close price is index 3
plt.plot(range(30, 130), pred_100[:, 3], label='Predicted Close', color='orange')
plt.axvline(29, color='gray', linestyle='--')
plt.title('Real (30) and Predicted (100) Close Prices')
plt.xlabel('Hour Index')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
